In [1]:
from openai import OpenAI
client = OpenAI(
    api_key = "",
    base_url="https://generativelanguage.googleapis.com/v1beta/"
)


response = client.chat.completions.create(
    model="gemini-1.5-flash",
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "Explain to me how AI works"
        }
    ]
)

print(response.choices[0].message)

ChatCompletionMessage(content='Artificial intelligence (AI) is a vast and complex field, but at its core, it\'s about creating systems that can perform tasks that typically require human intelligence.  This includes things like learning, problem-solving, decision-making, perception, and understanding language.  There\'s no single "how it works" answer, as different AI approaches use different techniques. However, we can break down the common principles:\n\n**1. Data is King:**  AI systems learn from data.  The more data you feed them, the better they generally perform. This data can be anything from images and text to sensor readings and financial transactions.  The quality and quantity of this data are crucial.  Garbage in, garbage out is a common saying in AI.\n\n**2. Algorithms are the Engine:**  Algorithms are sets of instructions that tell the computer how to process the data.  These algorithms are designed to extract patterns, make predictions, and improve their performance over 

In [2]:
from pydantic import BaseModel
from typing import Optional, Literal

class ParameterProperty(BaseModel):
    type: str
    description: str
    enum: Optional[list[str]] = None


class Parameters(BaseModel):
    type: Literal["object"]
    properties: dict[str, ParameterProperty]
    required: Optional[list[str]]


class Function(BaseModel):
    name: str
    description: str
    parameters: Parameters


class Tool(BaseModel):
    type: Literal["function"]
    function: Function

In [3]:
tools = [Tool(
    type="function",
    function=Function(
        name="search_weaviate",
        description="Search a Database containing information about Weaviate.",
        parameters=Parameters(
            type="object",
            properties={
                "search_query": ParameterProperty(
                    type="string",
                    description="The natural language query to search for in the database"
                )
            },
            required=["search_query"]
        )
    )
)]

In [7]:
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant. Use the supplied tools to assist the user."
    },
    {
        "role": "user",
        "content": "What did Weaviate release in 1.27?"
    }
]

def generate(lm_client, messages, tools):
    return lm_client.chat.completions.create(
        model="gemini-1.5-pro",
        messages=messages,
        tools=tools
    ).choices[0]


response = generate(client, messages, tools)

print(response)

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='model', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='0', function=Function(arguments='{"search_query":"What was released in Weaviate v1.27?"}', name='search_weaviate'), type='function')]))
